# Tracing and Observability

AutoGen has built-in support for tracing and observability for a collecting comprehensive record on the execution of your application. This is useful for debugging, performance analysis, and understanding the flow of your application. 

This feature is powered by the [opentelemetry](https://opentelemetry.io/) library,  meaning you can use any OpenTelemetry-compatible backend to collect and analyze the traces.


To begin we need to install the OpenTelemetry Python package. You can do this using pip:

```bash
pip install opentelemetry-sdk  
```

Once you have the skd installed, the simplest way to set up tracing in AutoGen is to:

- Configure an OpenTelemetry tracer provider
- Set up an exporter to send traces to your backend
- Connect the tracer provider to the AutoGen runtime

## Telemetry Backend

To collect and view traces, you need to set up a telemetry backend. There are several open source available, including Jaeger, Zipkin, and Prometheus. For this example, we will use Jaeger as our telemetry backend. 

For a quick start, you can run Jaeger locally using Docker:

```bash
docker run -d --name jaeger \
  -e COLLECTOR_OTLP_ENABLED=true \
  -p 16686:16686 \
  -p 4317:4317 \
  -p 4318:4318 \
  jaegertracing/all-in-one:latest
```

This command will start a Jaeger instance that listens on port 16686 for the Jaeger UI and port 4317 for the OpenTelemetry collector. You can access the Jaeger UI at `http://localhost:16686`.


## Instrumenting an AgentChat Team 

In the following section we will review how to enable tracing with an AutoGen GroupChat team.


In [1]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_core import SingleThreadedAgentRuntime
from autogen_ext.models.openai import OpenAIChatCompletionClient
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor


# Tracing setup.
def configure_oltp_tracing():
    jaeger_exporter = OTLPSpanExporter(endpoint="http://localhost:4317", insecure=True)
    tracer_provider = TracerProvider(resource=Resource({"service.name": "autogen-test-agentchat"}))
    span_processor = BatchSpanProcessor(jaeger_exporter)
    tracer_provider.add_span_processor(span_processor)
    trace.set_tracer_provider(tracer_provider)
    return tracer_provider


def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100


async def main():
    model_client = OpenAIChatCompletionClient(model="gpt-4o")

    planning_agent = AssistantAgent(
        "PlanningAgent",
        description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
        model_client=model_client,
        system_message="""
        You are a planning agent.
        Your job is to break down complex tasks into smaller, manageable subtasks.
        Your team members are:
            WebSearchAgent: Searches for information
            DataAnalystAgent: Performs calculations

        You only plan and delegate tasks - you do not execute them yourself.

        When assigning tasks, use this format:
        1. <agent> : <task>

        After all tasks are complete, summarize the findings and end with "TERMINATE".
        """,
    )

    web_search_agent = AssistantAgent(
        "WebSearchAgent",
        description="An agent for searching information on the web.",
        tools=[search_web_tool],
        model_client=model_client,
        system_message="""
        You are a web search agent.
        Your only tool is search_tool - use it to find information.
        You make only one search call at a time.
        Once you have the results, you never do calculations based on them.
        """,
    )

    data_analyst_agent = AssistantAgent(
        "DataAnalystAgent",
        description="An agent for performing calculations.",
        model_client=model_client,
        tools=[percentage_change_tool],
        system_message="""
        You are a data analyst.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
        If you have not seen the data, ask for it.
        """,
    )

    text_mention_termination = TextMentionTermination("TERMINATE")
    max_messages_termination = MaxMessageTermination(max_messages=25)
    termination = text_mention_termination | max_messages_termination

    selector_prompt = """Select an agent to perform task.

    {roles}

    Current conversation context:
    {history}

    Read the above conversation, then select an agent from {participants} to perform the next task.
    Make sure the planner agent has assigned tasks before other agents start working.
    Only select one agent.
    """

    task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"
    tracer_provider = configure_oltp_tracing()
    runtime = SingleThreadedAgentRuntime(tracer_provider=tracer_provider)
    tracer = trace.get_tracer("autogen-test-agentchat")
    with tracer.start_as_current_span("runtime"):
        runtime.start()
        team = SelectorGroupChat(
            [planning_agent, web_search_agent, data_analyst_agent],
            model_client=model_client,
            termination_condition=termination,
            selector_prompt=selector_prompt,
            allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
            runtime=runtime,
        )
        await Console(team.run_stream(task=task))

    await runtime.close()
    await model_client.close()


# asyncio.run(main())

In [3]:
await main()

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- PlanningAgent ----------
To find the required information, we need to perform the following tasks:

1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent: Search for this player's total rebounds in the 2007-2008 season.
3. WebSearchAgent: Search for this player's total rebounds in the 2008-2009 season.
4. DataAnalystAgent: Calculate the percentage change in the player's total rebounds between the 2007-2008 and 2008-2009 seasons.

Let's begin:

1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 season.
---------- WebSearchAgent ----------
[FunctionCall(id='call_VjKNxIR10LA5d66aWZtmzBhQ', arguments='{"query":"Miami Heat highest points scorer 2006-2007 season"}', name=

You can then use the Jaeger UI to view the traces collected by your application.  

![Jaeger UI](jaeger.png)

All of the code above which we have referenced in creating a SelectorGroupChat team should already be familiar to you. The important addition here is that we explicitly define a runtime which is configured with a trace provider, and the runtime is passed to the GroupChat team.

Underneath, the runtime is instrumented to log [messaging events](https://github.com/microsoft/autogen/blob/main/python/packages/autogen-core/src/autogen_core/_telemetry/_tracing_config.py) including:
- **create**: When a message is created
- **send**: When a message is sent
- **publish**: When a message is published
- **receive**: When a message is received
- **intercept**: When a message is intercepted
- **process**: When a message is processed
- **ack**: When a message is acknowledged

## Custom Traces 

So far, we are logging only the default events that are generated by the AutoGen runtime. However, you can also create custom spans to log specific events in your application. 

...

TBD: how do we log custom events like LLMCallEvents and ToolCallEvents? Are these already instrumented? Currenltly tool calls and their args are alredy logged (not sure where this is done), but LLM call events (iput and response are not .. posibly due to privacy?)